In [4]:
import nibabel as nib
import numpy as np
import gdist
import cython
import scipy
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from scipy import spatial
%matplotlib inline

In [5]:
### lose the subcortex now
reg_sphere=nib.load('LeftStructMSM/LeftStructMSMsphere.reg.surf.gii')
L_sphere=nib.load('./spheres/Sphere.ICO5.L.surf.gii')
subC=np.where(nib.load('../subcort/L.hum.SubC.ICO5.func.gii').darrays[0].data !=0)[0]


In [9]:
reg_coords=reg_sphere.darrays[0].data
Lcoords=L_sphere.darrays[0].data
verts=np.asarray(range(len(reg_coords)))


In [12]:
# verts=np.delete(verts,subC).tolist()

In [13]:
def find_corrVertex(A,B):
    ### where is a an x y z coordinate, and B is a numpy array 
    vert=np.argmin(np.sum(np.abs(np.subtract(A,B)),axis=1))
    return vert


In [14]:
### get correspondig vertex from pig for each point in human from MSM registration
new_verts=[]
for i in verts:
    new_verts.append(find_corrVertex(reg_coords[i],Lcoords))

In [15]:
### remove subcortex vertices prior to calculating MM index

#### mask out at end just priot to calculating mm index
# verts=np.delete(verts,subC)
# new_verts=np.delete(new_verts,subC)

In [16]:
##### gets the gradient profiles for the multimodal homology index
def get_gradient_profile(vertex,sphere,grads,nodist=True):
    src = np.array([vertex], dtype=np.int32)
    faces=nib.load(sphere).darrays[1].data.astype(np.int32)
    sphere=nib.load(sphere).darrays[0].data.astype(np.float64) 
    
    grads=nib.load(grads).agg_data()
    
    subC=np.where(grads[0] ==1)[0]
    
    
    if vertex in subC:
        #### don't calculate vertex present in sub cortex
        GradProfile=np.nan
    else:
        data=[]
        data=np.asarray(grads[1:])
        #### data finished loading
        #### calculate distance from vertex
        #### run on single vertex - reference gradient profile
        if nodist==True:
            GradProfile=data[:,src]
        else:
        #### calculate geodesic distance and find max prob
            vertsGeo=gdist.compute_gdist(sphere,faces,source_indices=src,target_indices=None,max_distance=12)
            vertsGeo=np.where(vertsGeo!=np.inf)[0].tolist()
            ROI=[i for i in vertsGeo if i not in subC]
            ROI=np.asarray(ROI)
    
            GradProfile=data[:,ROI].transpose()

    return GradProfile


In [19]:
def get_sim(H,P):
    sim=[]
    #### return nan if vertex corresponds to either species subcortex mask
    if type(P) ==float or type(H) ==float:
        sim=np.nan
    else:
        
        for i in range(len(P)):
            #### not this use case. only for when testing vertex to vertex and no sptial constraint
            if P.shape==H.shape:
                cos_sim=1-pairwise_distances(H.transpose(),
                                             P.transpose(), metric = 'cosine')
                sim.append(cos_sim)
            else:
                #### calculate between H and P wher P contains X vertices wihtin 8 mm of it and their gradient profiles
                cos_sim=1-pairwise_distances(H.transpose(),
                                             P[i].reshape(-1,1).transpose(),metric = 'cosine')
            sim.append(cos_sim)
        if len(sim) >=1:
            sim=np.stack(sim).flatten()
        else:
            sim=np.asarray(sim)
    return sim
        

In [35]:
#### build multimodal homology index
def max_hom(A):
    sim=[]
    if type(A) ==float:
        sim=np.nan
        return sim
    else:
        return np.max(A)

In [38]:
#### example cells for Ting
testH=get_gradient_profile(0,'./spheres/Sphere.ICO5.L.surf.gii',
                             'LeftStructMSM/L15humgrads.func.gii',nodist=True)
testP=get_gradient_profile(new_verts[0],'./spheres/Sphere.ICO5.L.surf.gii',
                             'LeftStructMSM/L15piggrads.func.gii',nodist=False)
print(f'Human gradient profile has shape {testH.shape}, \
and the pig with 12mm geodesic distance has {testP.shape} \n')
sim=get_sim(testH,testP)
print(f'The similarity of gradient profiles for each vertex in 12mm of our vertex is: \n {sim}\n')
hom=max_hom(sim)
print(f'the max homology of this vertex is {hom}')



Human gradient profile has shape (15, 1), and the pig with 12mm geodesic distance has (20, 15) 

The similarity of gradient profiles for each vertex in 12mm of our vertex is: 
 [-0.22034526 -0.25886595 -0.28650832 -0.23610044 -0.26376855 -0.284261
 -0.27253246 -0.23834896 -0.27028108 -0.28428864 -0.28486383 -0.30051506
 -0.2669921  -0.28284132 -0.29017377 -0.30220842 -0.27929878 -0.28727663
 -0.29995692 -0.2848922 ]

the max homology of this vertex is -0.22034525871276855


In [22]:
SFCindex=[]
for i in range(len(verts)):
    H=get_gradient_profile(verts[i],'./spheres/Sphere.ICO5.L.surf.gii','LeftStructMSM/L15humgrads.func.gii', nodist=True)
    P=get_gradient_profile(new_verts[i],'./spheres/Sphere.ICO5.L.surf.gii', 'LeftStructMSM/L15piggrads.func.gii',nodist=False)
    SFCindex.append(max_hom(get_sim(H,P)))

In [24]:
def save_gifti(data,out):
    gi = nib.gifti.GiftiImage()
    da = nib.gifti.GiftiDataArray(np.float32(data))
    gi.add_gifti_data_array(da)
    nib.save(gi,f'{out}.func.gii')
save_gifti(np.asarray(SFCindex),'StructFCHomologyIndex')